<font color='blue'> **SFR** </font>

In [7]:
import os; import numpy as np ; import pandas as pd ; import rasterio  ; from rasterio.plot import show  ; from rasterio.merge import merge ; import geopandas as gp ; from pyproj import CRS
from pathlib import Path ; import openpyxl ; import matplotlib.pyplot as plt ; import mfexport  ; import sys ; sys.path.append('../python_packages_static') ; import flopy as fp    
from shapely.geometry import shape  ; from gisutils import project
TA=Path("./TA");TB=Path("./TB");TC=Path("./TC")   ; TD=Path("./TD")            ; TE=Path("./TE");TE.mkdir(exist_ok=True)          ; import fiona
PP = Path('E:/15_REPOS/05_Working/01_nerversink') ; PB=Path(PP/'PB')           ; PC=Path(PP/'PC')

### Get buffered bounding box around model domain
uses `gis-utils`: https://github.com/aleaf/gis-utils

In [8]:
extent_poly     = shape(fiona.open(PC/'extend_15km_buffer.shp').next()['geometry'])           ; epsg = 5070
extent_poly_ll = project(extent_poly, "+init=epsg:{}".format(epsg), "+init=epsg:4269")    ; extent_poly_ll.bounds

C:\Users\gross.DESKTOP-AIRQM2M\AppData\Local\Temp\ipykernel_16432\3984914174.py:1: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  extent_poly     = shape(fiona.open(PC/'extend_15km_buffer.shp').next()['geometry'])           ; epsg = 5070
c:\Users\gross.DESKTOP-AIRQM2M\anaconda3\envs\mfsetup\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


(-74.69929614110991,
 41.515449698651224,
 -74.32569021016936,
 41.862482583960265)

### Pull NWIS streamflow daily value sites within bounding box
uses `pydrograph`: https://github.com/aleaf/pydrograph

In [ ]:
nwis = Nwis(extent=extent_poly_ll)

In [ ]:
all_dvs = nwis.get_siteinfo('daily_values')

In [ ]:
# look at NWIS sites with daily values within bounding box -- 7 sites total. 
all_dvs

## Gage 01436000 located below Neversink Reservoir, use for SFR Inflow
pull all daily values from this site

In [ ]:
neversink_inflow = nwis.get_all_dvs(['01436000'])

In [ ]:
neversink_inflow = neversink_inflow['01436000']
neversink_inflow = neversink_inflow.rename(columns={'105408_00060_00003':'discharge_cfs'})

In [ ]:
neversink_inflow.head()

### Want the SFR inflow data set period of record to match the period of record at the downstream gage (01436500, Neversink near Woodbourne, NY) used for flux target (1954-1993). 
The Neversink Reservoir dam was completed late 1953. Downstream flux target period of record ends at WY 1993.
**Use median flow 1954-1993 for SFR inflow**

In [ ]:
plt.figure(figsize=(8,6))
neversink_inflow[['discharge_cfs']].plot()
plt.ylabel('01436000 discharge, in cfs')
plt.axvline(pd.to_datetime('10/1/1953'), ls='--', c='black', label='Dam built')
plt.axvline(pd.to_datetime('09-30-1993'), ls='--', c='orange', label='01436500 record ends')
plt.legend()
plt.show()

In [ ]:
m3d_neversink_inflow = neversink_inflow.loc['10-01-1954':'09-30-1993']['discharge_cfs'].quantile(.5) * 2446.58  #  convert from cfs to cubic meters per day

print('neversink inflow: {0:.1f} cfs, {1:.0f} m3/d'.format(
    neversink_inflow.loc['10-01-1954':'09-30-1993']['discharge_cfs'].quantile(.5), 
    m3d_neversink_inflow
    ))

In [ ]:
inflow_df = pd.DataFrame({'line_id':[200105086],
                         'flow_m3d':[m3d_neversink_inflow],
                         'category':['NWIS']})

inflow_df

In [ ]:
inflow_df.to_csv('../processed_data/neversink_inflow.csv',index=None)